In [1]:
import pandas as pd
import numpy as np
# 加载Excel文件
xls = pd.ExcelFile("E:\\数学建模国赛2023选题\\C题\\第二题建模约束参数.xlsx")


In [2]:
# 查看Excel文件中各个sheet的前几行数据以了解结构和列名
sheet_names = xls.sheet_names

# 存储每个sheet的前几行数据
sheet_previews = {}

for sheet in sheet_names:
    sheet_previews[sheet] = pd.read_excel(xls, sheet).head()

sheet_previews


{'Model_Params':            a         b   星期   分类名称
 0  84.277492  0.974480  星期三  水生根茎类
 1  81.092421  1.009424  星期三    花叶类
 2  14.822394  1.011291  星期三    花菜类
 3  25.241518  0.999187  星期三     茄类
 4  61.474034  1.000100  星期三    辣椒类,
 'Markup_Bounds':     分类名称  lower_bound  upper_bound
 0  水生根茎类    44.059830    96.064736
 1    花叶类    55.054544    97.460568
 2    花菜类    36.620075    83.754887
 3     茄类    43.469963   134.360511
 4    辣椒类    46.272717   148.951008,
 'Sales_Bounds':     分类名称  lower_bound  upper_bound
 0  水生根茎类      4.82050     91.51175
 1    花叶类    105.64975    289.32900
 2    花菜类      8.09800     79.35975
 3     茄类     12.86075     51.80800
 4    辣椒类     33.14950    151.92225,
 'Daily_Sales_Bounds':    lower_bound  upper_bound
 0    228.75775    650.33475}

In [3]:
# 重新读取Excel文件中的各个表格，使用正确的工作表名称
model_params_df = pd.read_excel(xls, 'Model_Params')
markup_bounds_df = pd.read_excel(xls, 'Markup_Bounds')
sales_bounds_df = pd.read_excel(xls, 'Sales_Bounds')
daily_sales_bounds_df = pd.read_excel(xls, 'Daily_Sales_Bounds')

# 从model_params表格中筛选出星期一的数据，并提取相应的a和b值
model_params_week1 = model_params_df[model_params_df['星期'] == '星期一']
a_values_week1 = model_params_week1['a'].values
b_values_week1 = model_params_week1['b'].values

# 从其他表格中提取约束参数
markup_lower_bound = markup_bounds_df['lower_bound'].values
markup_upper_bound = markup_bounds_df['upper_bound'].values
sales_lower_bound = sales_bounds_df['lower_bound'].values
sales_upper_bound = sales_bounds_df['upper_bound'].values
daily_sales_lower_bound = daily_sales_bounds_df.loc[0, 'lower_bound']
daily_sales_upper_bound = daily_sales_bounds_df.loc[0, 'upper_bound']

a_values_week1, b_values_week1, markup_lower_bound, markup_upper_bound, sales_lower_bound, sales_upper_bound, daily_sales_lower_bound, daily_sales_upper_bound


(array([103.48582135,  78.8282481 ,  13.51245783,  28.86845876,
         54.17890353,  38.8385542 ]),
 array([0.97243359, 1.00989859, 1.01349253, 0.99879302, 1.00122423,
        0.99955113]),
 array([44.05983035, 55.05454389, 36.62007474, 43.46996296, 46.27271659,
        45.74687858]),
 array([ 96.06473596,  97.4605679 ,  83.7548875 , 134.36051093,
        148.95100802,  89.77916031]),
 array([  4.8205 , 105.64975,   8.098  ,  12.86075,  33.1495 ,  24.31175]),
 array([ 91.51175, 289.329  ,  79.35975,  51.808  , 151.92225,  72.90125]),
 228.75775,
 650.33475)

In [4]:
from pyswarm import pso

# 定义目标函数（需要最大化，因此取负数进行最小化）
def objective(vars):
    x = vars[:6]
    y = vars[6:]
    return -np.sum(x * y)

# 定义约束函数（返回值应为正数或零）
def constraints(vars):
    cons = []

    # 添加6个模型约束（只针对星期一）
    epsilon = 0.00001# 选择一个合适的epsilon值
    for i in range(6):
        cons.append(abs(vars[6+i] - a_values_week1[i] * np.exp(b_values_week1[i] * vars[i])) - epsilon)

    # 添加12个范围约束（6个对于成本加价，6个对于销量）
    for i in range(6):
        cons.append(vars[i] - markup_lower_bound[i])
        cons.append(markup_upper_bound[i] - vars[i])
        cons.append(vars[6+i] - sales_lower_bound[i])
        cons.append(sales_upper_bound[i] - vars[6+i])

    # 添加每日销售总量的上下界约束（针对星期一）
    cons.append(daily_sales_upper_bound - np.sum(vars[6:]))
    cons.append(np.sum(vars[6:]) - daily_sales_lower_bound)

    return cons

# 设置变量界限
lb = list(markup_lower_bound) + list(sales_lower_bound)
ub = list(markup_upper_bound) + list(sales_upper_bound)

# 运行粒子群优化
xopt, fopt = pso(objective, lb, ub, f_ieqcons=constraints)

# 打印结果
print("Optimized variables are:", xopt)
print("Optimized objective is:", -fopt)


Stopping search: maximum iterations reached --> 100
Optimized variables are: [ 96.06473025  97.4605679   83.75488168 134.3587258  148.95100802
  89.77916031  64.27655583 289.32711659  37.01173043  34.89623493
 151.92225     72.90085064]
Optimized objective is: 71335.17128608088


In [5]:
xopt, fopt

(array([ 96.06473025,  97.4605679 ,  83.75488168, 134.3587258 ,
        148.95100802,  89.77916031,  64.27655583, 289.32711659,
         37.01173043,  34.89623493, 151.92225   ,  72.90085064]),
 -71335.17128608088)